In [2]:
! pip3 install -qU langchain-upstage

In [3]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [4]:
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Quick hello world
from langchain_upstage import ChatUpstage

llm = ChatUpstage()
llm.invoke("What's the best season to get to Korean?")

AIMessage(content="The best season to learn Korean is whenever you feel ready and motivated to start. Learning a new language is a personal journey, and there's no perfect time or season to begin. The most important factor is your willingness to learn and dedicate time and effort to the process.", response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 21, 'total_tokens': 82}, 'model_name': 'solar-1-mini-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c58541a1-09b9-41f1-a0d9-9be03cbde84f-0')

In [10]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "What is the capital of France?"),
        ("ai", "I know of it. It's Paris!!"),
        ("human", "What about Korea?"),
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()
chain.invoke({})



"Ah, Korea! I can tell you that the capital of South Korea is Seoul! And the capital of North Korea is Pyongyang. I'm pretty good, huh? Now, what about the capital of England?"

In [22]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'29'

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'29.'

![Zero-Shot COT](figures/zero-cot.webp)
From https://arxiv.org/abs/2205.11916

In [25]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'Step 1: Determine the number of apples used for lunch. The cafeteria used 20 apples.\nStep 2: Subtract the apples used for lunch from the initial number of apples. The cafeteria had 23 apples, so 23 - 20 = 3 apples remaining.\nStep 3: Add the number of apples bought to the remaining apples. The cafeteria bought 6 more apples, so 3 + 6 = 9 apples.\n\nA: The cafeteria has 9 apples now.'

![CoT](figures/cot.webp)

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria had 23 apples. They used 20 to make lunch, leaving 3 apples. They then bought 6 more apples, bringing the total to 9 apples. The answer is 9.'

## divide and conquer
![divideandconquer](figures/divideandconquer.png)

In [39]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. What is SOLAR 10.7B, and what are its key features?\n2. How is SOLAR 10.7B different from other large language models?\n3. What is depth up-scaling (DUS), and how does it contribute to scaling LLMs?'

In [30]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'1. SOLAR 10.7B: This is the name of the large language model introduced in the text.\n2. Depth up-scaling (DUS): This is the method used for scaling the LLM, which includes depthwise scaling and continued pretraining.\n3. Instruction-following capabilities: This refers to the fine-tuned variant of SOLAR 10.7B that has improved capabilities for following instructions.'

In [38]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "Depth up-scaling (DUS)":
    
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'What is the difference between depth up-scaling (DUS) and other LLM up-scaling methods that use mixture-of-experts?'

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [12]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

'Why did the chicken join the band? Because it had the drums!'

In [14]:
chain.invoke({"adjective": "funny", "content": "beef"})

'Why did the beef cross the road?\n\nTo get to the other side of the grill!'

In [40]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "{keyword}":
    
    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the method for scaling large language models (LLMs) called, which encompasses depthwise scaling and continued pretraining, and does not require complex changes to train and inference efficiently?'

# Excercise 

Complete this code to create excellent questions from given documents using the following steps:
1. Include Few Shot examples, including CoT.
2. Generate keywords and topics first.
3. Iterate through each keyword and topic to generate questions.
4. Compare the generated questions with zero-shot generated questions.
